<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">LEAK DATA </h1>
<br>


# Add Libaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load and Examine Data

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv") #loading datas
test_df = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

In [ ]:
train_df.head(3)

In [ ]:
train_df.info()

In [ ]:
print(f"train data len : {len(train_df)}")
print(f"test data len : {len(test_df)}")

In [ ]:
y_train = train_df.loc[:,"loss"]
x_train = train_df.drop(["id","loss"],axis=1)

x_test_id = test_df.loc[:,"id"]
x_test = test_df.drop(["id"],axis=1)

con_x_data = pd.concat([x_train,x_test],axis = 0).reset_index(drop = True)

In [ ]:
scaler = StandardScaler()

con_x_data = scaler.fit_transform(con_x_data)

In [ ]:
x_test = con_x_data[250000:]
x_train = con_x_data[:250000]

In [ ]:
y_train = y_train.to_numpy()

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

In [ ]:
def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [ ]:
feat=backward_elimination(x_train,y_train)

x_train = x_train.loc[:,feat]

In [ ]:
model = sm.OLS(y_train,x_train).fit()
model.summary()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
gb_model = GradientBoostingRegressor(subsample=0.7,learning_rate=0.017, max_depth=4 ,
                                     random_state=np.random.randint(1000),n_estimators=800).fit(X_train, y_train)

print("Performance on train data:", gb_model.score(X_train, y_train))